# CARGUE DE LOS DATOS Y TRANSFORMACIONES

In [1]:
pip install sodapy


Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install xgboost


Note: you may need to restart the kernel to use updated packages.


In [3]:
from sodapy import Socrata
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score

import warnings
warnings.filterwarnings('ignore')

In [4]:
client = Socrata('www.datos.gov.co', None)
tempTr = client.get("8yi9-t44c", limit=200000)
datos = pd.DataFrame.from_records(tempTr) 
datos.head()

,idpeaje,peaje,categoriatarifa,desde,hasta,valortarifa,cantidadtrafico,cantidadevasores,cantidadexentos787
0,1,ALVARADO,I,2015-08-21T00:00:00.000,2015-08-31T00:00:00.000,7000,27730,0,660
1,1,ALVARADO,II,2015-08-21T00:00:00.000,2015-08-31T00:00:00.000,7600,9930,0,70
2,1,ALVARADO,III,2015-08-21T00:00:00.000,2015-08-31T00:00:00.000,16100,1277,0,0
3,1,ALVARADO,IV,2015-08-21T00:00:00.000,2015-08-31T00:00:00.000,20400,2074,0,0
4,1,ALVARADO,V,2015-08-21T00:00:00.000,2015-08-31T00:00:00.000,22900,4167,0,0


# LIMPIEZA Y ALISTAMIENTO

In [5]:
datos.desde = pd.to_datetime(datos.desde)
datos.hasta = pd.to_datetime(datos.hasta)
datos.valortarifa = datos.valortarifa.astype('int')
datos.cantidadtrafico = datos.cantidadtrafico.astype('int')
datos.cantidadevasores = datos.cantidadevasores.apply(pd.to_numeric, errors='coerce')
datos.cantidadexentos787 = datos.cantidadexentos787.apply(pd.to_numeric, errors='coerce')
datos['año'] = pd.DatetimeIndex(datos['hasta']).year
datos['mes'] = pd.DatetimeIndex(datos['hasta']).month
datos['añomes'] = pd.to_datetime(datos.hasta).dt.to_period('M')
datos['recaudo'] = datos.valortarifa * datos.cantidadtrafico

datos.head()

,idpeaje,peaje,categoriatarifa,desde,hasta,valortarifa,cantidadtrafico,cantidadevasores,cantidadexentos787,año,mes,añomes,recaudo
0,1,ALVARADO,I,2015-08-21,2015-08-31,7000,27730,0.0,660.0,2015,8,2015-08,194110000
1,1,ALVARADO,II,2015-08-21,2015-08-31,7600,9930,0.0,70.0,2015,8,2015-08,75468000
2,1,ALVARADO,III,2015-08-21,2015-08-31,16100,1277,0.0,0.0,2015,8,2015-08,20559700
3,1,ALVARADO,IV,2015-08-21,2015-08-31,20400,2074,0.0,0.0,2015,8,2015-08,42309600
4,1,ALVARADO,V,2015-08-21,2015-08-31,22900,4167,0.0,0.0,2015,8,2015-08,95424300


In [6]:
cat = LabelEncoder().fit(datos.categoriatarifa)
datos.categoriatarifa = cat.fit_transform(datos.categoriatarifa)

In [ ]:


# Supongamos que datos es tu DataFrame original
# Codificar columnas categóricas si es necesario
le = LabelEncoder()
datos['idpeaje'] = le.fit_transform(datos['idpeaje'])




In [7]:
datos.cantidadevasores.fillna(0,inplace=True)
datos.cantidadexentos787.fillna(0,inplace=True)

In [8]:
pd.isna(datos).sum()

idpeaje               0
peaje                 0
categoriatarifa       0
desde                 0
hasta                 0
valortarifa           0
cantidadtrafico       0
cantidadevasores      0
cantidadexentos787    0
año                   0
mes                   0
añomes                0
recaudo               0
dtype: int64

In [9]:
datosT = datos.groupby(by=['idpeaje', 'peaje', 'categoriatarifa', 'valortarifa', 'año', 'mes', 'añomes'], as_index=False).sum()


TypeError: datetime64 type does not support sum operations

In [10]:
columnasX = ['idpeaje','categoriatarifa','año','mes','valortarifa','cantidadevasores','cantidadexentos787']
columnasY = ['cantidadtrafico']

In [11]:
Xtrain, Xtest, ytrain, ytest = train_test_split(datos[columnasX], datos[columnasY], test_size=0.30, random_state=8)

# REVISIÓN OPCIONES DE MODELOS

## Regresión logística

In [ ]:
from sklearn.feature_selection import SelectKBest, f_regression

# Elegir el número de características que deseas mantener (asegúrate de que no sea más grande que el número total de características)
num_features_to_keep = 7  # Puedes ajustar este número según sea necesario

# Aplicar SelectKBest para seleccionar las mejores características
selector = SelectKBest(f_regression, k=num_features_to_keep)
Xtrain_reduced = selector.fit_transform(Xtrain, ytrain)
Xtest_reduced = selector.transform(Xtest)

# Crear y ajustar el modelo de regresión logística con las características reducidas
regLog = LogisticRegression()
regLog.fit(Xtrain_reduced, ytrain)
predRegLog = regLog.predict(Xtest_reduced)


In [12]:
regLog = LogisticRegression()
regLog.fit(Xtrain,ytrain)
predRegLog = regLog.predict(Xtest)
predRegLog

MemoryError: Unable to allocate 17.7 GiB for an array with shape (80357, 29565) and data type float64

In [ ]:
# Calcular MSE y R2
mse = mean_squared_error(ytest, predRegLog)
r2 = r2_score(ytest, predRegLog)

# Imprimir resultados
print(f'Mean Squared Error (MSE): {mse}')
print(f'Coeficiente de Determinación (R2): {r2}')

## K vecinos

In [ ]:
regKN = KNeighborsRegressor()
regKN.fit(Xtrain,ytrain)
predRegKN = regKN.predict(Xtest)
predRegKN

## Random Forest

In [ ]:
regRF = RandomForestRegressor()
regRF.fit(Xtrain,ytrain)
predRegRF = regRF.predict(Xtest)
predRegRF

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBRegressor

# Supongamos que datos es tu DataFrame original
# Codificar columnas categóricas si es necesario
le = LabelEncoder()
datos['idpeaje'] = le.fit_transform(datos['idpeaje'])

# Dividir los datos en características (X) y columna de destino (y)
X = datos.drop('columna_objetivo', axis=1)  # Ajusta 'columna_objetivo' a tu columna de destino real
y = datos['columna_objetivo']

# Dividir los datos en conjuntos de entrenamiento y prueba
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear y entrenar el modelo XGBRegressor
regXGB = XGBRegressor()
regXGB.fit(Xtrain, ytrain)

# Realizar predicciones
predRegXGB = regXGB.predict(Xtest)


## XGBoost

In [ ]:
regXGB = XGBRegressor()
regXGB.fit(Xtrain,ytrain)
predRegXGB = regXGB.predict(Xtest)
predRegXGB

# MODELOS ESCOGIDOS